In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_excel('data/data.xlsx', index_col='Unnamed: 0')

In [ ]:
# т. к. пустых значений не много, а заполнять их не представляется возможным - удаляем их
df.dropna(how='any', inplace=True)

In [ ]:
# Из EDA мы помним о наличии больших выбросов. Уберем их
q_low, q_high = df["SI"].quantile([0.01, 0.99])
df_filtered = df[(df["SI"] >= q_low) & (df["SI"] <= q_high)]

# Создадим столбец, который будет содержать 2 поля: 1 если значение превышает медиану и 
# 0 в обратном случае

df_filtered['Class_4'] = [np.nan]*len(df_filtered)
df_filtered.loc[df_filtered['SI']>8, 'Class_4'] = 1
df_filtered.loc[df_filtered['SI']<=8, 'Class_4'] = 0

X = df_filtered.drop(['IC50, mM', 'CC50, mM', 'SI', 'Class_1', 'Class_2', 'Class_3', 'Class_4'], axis=1)
y = df_filtered['Class_4']

# Классификация: превышает ли значение SI значение 8

In [ ]:
# Из EDA мы помним о наличии больших выбросов. Уберем их
q_low, q_high = df["SI"].quantile([0.01, 0.99])
df_filtered = df[(df["SI"] >= q_low) & (df["SI"] <= q_high)]

# Создадим столбец, который будет содержать 2 поля: 1 если значение превышает медиану и 
# 0 в обратном случае

df_filtered['Class_4'] = [np.nan]*len(df_filtered)
df_filtered.loc[df_filtered['SI']>8, 'Class_4'] = 1
df_filtered.loc[df_filtered['SI']<=8, 'Class_4'] = 0

X = df_filtered.drop(['IC50, mM', 'CC50, mM', 'SI', 'Class_1', 'Class_2', 'Class_3', 'Class_4'], axis=1)
y = df_filtered['Class_4']


C:\Users\raven\AppData\Local\Temp\ipykernel_2848\1619424459.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Class_4'] = [np.nan]*len(df_filtered)


Видим дизбаланс классов и запомним это

In [ ]:
# Отбор важных признаков
selector_forest = SelectFromModel(
    RandomForestClassifier(n_estimators=100),
    threshold="median"
)
selector_forest.fit(X, y)

selected_features = [
                        'MolLogP', 'TPSA', 'NumHDonors', 'NumHAcceptors', 
                        'fr_halogen', 'qed', 'FractionCSP3', 'SPS'
                    ]+list(X.columns[selector_forest.get_support()])
selected_features = list(set(selected_features))

print(f"Всего отобрано признаков: {len(selected_features)}")

Всего отобрано признаков: 107


In [ ]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X[selected_features], y, stratify=y, test_size=0.2, random_state=42
)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(
        class_weight='balanced', #Из-за дисбаланса классов
        random_state=42))
])

param_grid = {
    'clf__n_estimators': [100, 200, 300],
    'clf__max_depth': [5, 10, 15, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['sqrt', 'log2', None]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nМодель классификации случайного леса:")
print("Лучшие параметры:", grid_search.best_params_)
print(f"Основные метрики:\n {classification_report(y_test, y_pred)}")
print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred)}")

C:\Users\raven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,



Модель классификации случайного леса:
Лучшие параметры: {'clf__max_depth': 10, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 4, 'clf__min_samples_split': 10, 'clf__n_estimators': 100}
Основные метрики:
               precision    recall  f1-score   support

         0.0       0.76      0.75      0.75       127
         1.0       0.55      0.57      0.56        69

    accuracy                           0.68       196
   macro avg       0.65      0.66      0.66       196
weighted avg       0.69      0.68      0.68       196

Confusion Matrix:
 [[95 32]
 [30 39]]


In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 7],
    'learning_rate': [0.05, 0.1, 0.15],
    'gamma': [0, 0.1],
    'reg_alpha': [0, 0.1],
    'subsample': [0.8, 0.9], 
    'colsample_bytree': [0.8, 0.9]
}

xgb_classifier = XGBClassifier(
        objective='binary:logistic', # т. к. только 2 класса
        random_state=42,
        scale_pos_weight=sum(y_train==0)/sum(y_train==1),
    )

grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1,
)

# Запуск поиска
grid_search.fit(X_train, y_train)

y_pred = grid_search.predict(X_test)

print("\nМодель классификации XGB:")
print("Лучшие параметры:", grid_search.best_params_)
print(f"Основные метрики: {classification_report(y_test, y_pred)}")
print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")


Модель классификации XGB:
Лучшие параметры: {'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 50, 'reg_alpha': 0, 'subsample': 0.9}
Основные метрики:               precision    recall  f1-score   support

         0.0       0.77      0.69      0.72       127
         1.0       0.52      0.62      0.57        69

    accuracy                           0.66       196
   macro avg       0.64      0.65      0.65       196
weighted avg       0.68      0.66      0.67       196

Confusion Matrix: [[87 40]
 [26 43]]


C:\Users\raven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


В данном случае снова лучше оказалось модель леса, оставляем ее